In [7]:
# %load_ext autoreload
# %autoreload 2

import json
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import requests
import pandas as pd
import altair as alt

if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir('..')

plt.style.use('ggplot')
alt.themes.enable('fivethirtyeight')
CHARTS_DIR = Path('../covid19-analysis/layouts/partials/covid')

In [9]:
from fetch import fetch_timeseries, TS_URL

df = fetch_timeseries(TS_URL)
df_long = df.stack().rename('count').rename_axis(index={None: 'status'})

display(df.head())
display(df_long.head())

# Timeseries

confirmed  deaths  recovered
country     date                                    
Afghanistan 2020-01-22          0       0          0
            2020-01-23          0       0          0
            2020-01-24          0       0          0
            2020-01-25          0       0          0
            2020-01-26          0       0          0

country      date        status   
Afghanistan  2020-01-22  confirmed    0
                         deaths       0
                         recovered    0
             2020-01-23  confirmed    0
                         deaths       0
Name: count, dtype: int64

In [11]:
from IPython.display import display
from importlib import reload
import charts
import render
reload(charts)
reload(render)
from charts import make_ts_chart, make_ts_selections, make_data_long
from charts import make_map, status_schemes, make_map_data, countries, make_map_data, make_dod, combine_map_ts
from render import make_chart

# alt.data_transformers.enable('default', max_rows=None)
alt.data_transformers.enable('data_server')

make_chart(df_long)

In [4]:
df.loc['France'].reset_index().assign(date_int=lambda f: f['date'].dt.dayofyear)

,date,confirmed,deaths,recovered,date_int
0,2020-01-22,0,0,0,22
1,2020-01-23,0,0,0,23
2,2020-01-24,2,0,0,24
3,2020-01-25,3,0,0,25
4,2020-01-26,3,0,0,26
...,...,...,...,...,...
67,2020-03-29,40708,2611,7226,89
68,2020-03-30,45170,3030,7964,90
69,2020-03-31,52827,3532,9513,91
70,2020-04-01,57749,4043,11053,92


In [5]:
from charts import make_dod, assign_dod

dod_long = make_dod(df_long)
dod_long_world = df_long.groupby(['date', 'status']).sum().to_frame().groupby('status').apply(assign_dod)

In [11]:
# alt.Chart(dod_long_world.reset_index().dropna()).mark_bar().encode(x='date', y='diff_rel', color='status')
print('ok')

ok


In [12]:

# equirectangular', 'mercator', 'orthographic', 'gnomonic
# map_chart2.properties(width=900, title='Confirmed cases').project('mercator') & (ts_chart2 | make_dod_chart(dod_long))

In [8]:
confirmed_chart = alt.Chart(df.loc['Spain'].reset_index().assign(date_int=lambda f: f['date'].dt.dayofyear).dropna()).mark_point().encode(x='date_int', y='confirmed')
# confirmed_chart + confirmed_chart.transform_regression('date_int', 'confirmed', method='exp').mark_line()

In [23]:
# selection_legend = alt.selection_single(fields=['status'], bind='legend', empty='all')


# Correlations

In [15]:
import seaborn as sns

df.groupby('country').apply(lambda f: f.fillna(method='ffill')).fillna(0).corr().pipe(sns.heatmap, annot=True)

In [16]:
df.groupby(['date']).sum().corr()

,confirmed,deaths,recovered
confirmed,1.00000,0.997950,0.938290
deaths,0.99795,1.000000,0.924285
recovered,0.93829,0.924285,1.000000


In [17]:
def correlation_lags(df, column='deaths', max_lag=20, group=False):
    def series_corr(f):
        return pd.Series({t: f['confirmed'].shift(t).iloc[:-20].corr(f[column]) for t in range(max_lag)})
    if group:
        return df.groupby('country').apply(series_corr).idxmax(axis=1)
    else:
        return df.pipe(series_corr).idxmax(axis=0)

world_ts = df.groupby('date').sum()
days_to_death = correlation_lags(world_ts, 'deaths')
days_to_recov = correlation_lags(world_ts, 'recovered')

In [18]:
days_to_death, days_to_recov

(1, 19)

In [20]:
fatality_rates = (df.join(correlation_lags(df, group=True, column='deaths').rename('days_to_death').fillna(0).astype(int))
                  .groupby('country')
                  .apply(lambda g: g.deaths.div(g.confirmed.shift(g['days_to_death'].iloc[0])))
                  .reset_index(0, drop=True)
                 )
fatality_rates.groupby('date').mean().plot()

In [23]:
world_ts.deaths.div(world_ts.confirmed.shift(days_to_death)).plot()